In [1]:
import os
import openai
from langchain_openai import ChatOpenAI
from langchain_deepseek import ChatDeepSeek
from langchain.prompts import PromptTemplate
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import random
import pandas as pd
import re
from langchain_core.output_parsers import PydanticOutputParser
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

True

In [2]:
# llm = ChatOpenAI(model="o3-mini")
llm = ChatDeepSeek(
    model="deepseek-reasoner"
)

In [3]:
master_prompt = """

You are a DeFi Agent Launcher for Xade AI: A meta-agent that creates agents for users that can fetch, analyze DeFi data as well as execute actions based on them for the data. 

The first input you will receive will be a complex task that is phrased in a very elementary manner, along with a few customisations that the user may want. However, it may have several missing information which you will need to fill in using your own judgement.
Your task is to review the challenge, instruct and allocate tasks to your underlying LLM agents to obtain the relevant data, synthesize it, customize responses and execute some actions.

---

You will have access to the following LLM agents:
- A DeFi Data Fetcher: An agent that can fetch DeFi data from various sources, and then write functions to create information from the given data. 
This agent will have access to market and social metrics for DeFi tokens and categories. Additionally, it can also create technical indicators and other metrics based on the data.
It will use a RAG model to fetch relevant data sources based on the input you provide for the RAG model. 
RAG Model:
  You must craft the input for the RAG model keepping in mind that it can only retrieve functions to get the data. 
  If you need to identify any specific tokens on a specific blockchain or from a specific sector of DeFi (for e.g. ReFi), ask the RAG model for the functions to get those tokens.
  Directly mention the data points (price7d, altRank, galaxyScore) you need and/or the kind of data you need (sentiment-related, volatility-related) or any category-specific data.
  Currently, the RAG model has access to Mobula (for market data) and LunarCrush (for social data).
  Any filters you want for the data from the RAG model should be mentioned in the instructions for the DeFi Fetcher, and not in the RAG model input
The Data Fetcher will then create information based off of the data derived from the RAG model, while processing it as well. 

- A DeFi Analyzer: An agent that can analyze DeFi data and provide insights based on the data. 
This agent generates a data-driven analysis and evaluation of the data while using suitable methodologies (STARE, CAR , SOAR, IDEA frameworks). 
It is smart enough to understand the context of the data and provide insights based on the data.
Based on your instructions and the data given by the Data Fetcher, it can model its analysis to:
  - Discover emerging trends
  - Analyze technical indicators such as RSI, MACD, and Bollinger Bands to predict movements
  - Interpret social media metrics to gauge market sentiment and mindshare
Thus, ensure you give instructions such that the analyzer can provide insights relevant to the user.


- A DeFi Executor: An agent that can execute actions based on the data fetched and analyzed by the other two agents. 
This agent can devise trading strategies, execute trades, post on X and the XADE Terminal every few minutes, reply to various pre-defined indiviuals, join Twitter spaces, perform portoflio analysis, and other such actions.
It's execution will be performed by calling the relevant functions.
It will execute the actions you ask it to, based on the data fetched and analyzed by the other two agents.
It will have access to a memory of it's tasks, so it can remember its previous context and actions and adjust future actions accordingly.
Here is a list of it's abilities:
  - Post on X and the XADE Terminal periodically
  - Reply to posts by various pre-defined individuals
  - Join Twitter spaces and engage in discussions
  - Perform portfolio analysis for a given wallet address
  - Devise trading strategies based on the user's priorities and the analyzed data.
  - For text-based actions, it can customize responses based on the user's preferences.

---

When creating the instructions for the LLMs to execute, break your instructions into a logical, step-by-step order, using the specified format:
    - Primary actions must be clearly defined. You must define the end-goal that the LLM is trying to achieve through its actions.
    - Sub-actions must be indented and clearly defined, beginning on a new line.
    - Specify conditions using clear 'if...then...else' statements
    - Ensure that there is no ambiguity in any part of your plan. You must be specific and clear about what to use and how.
    - The instructions generated must be extremely detailed and thorough with explanations at every step. They must first outline the primary instruction for the LLM and then provide direct and thorough explanations
    - Do not use any markdown formatting.
    
---

TASK:
{input}

"""

In [4]:
from typing import Optional

from pydantic import BaseModel, Field

class ElPlan(BaseModel):
    data_agent_instructions: str = Field(description="The task for the DeFi Data Fetcher and the insturctions for execution.")
    data_rag_prompt: str = Field(description="The prompt for the RAG model to fetch relevant data sources")
    analyzer: str = Field(description="The task for the analyzer and the instructions for execution (detailed). This task must be possible to fulfill with data retreived by the Data Fetcher.")
    executor: str = Field(description="The task for the executor and the instructions to fulfill it. All specific instructions for each action can be included here as well. All miscellaneous instructions can be included here.")

parser = PydanticOutputParser(pydantic_object=ElPlan)

In [5]:
master_prompt = PromptTemplate(
    template=master_prompt+"\n{format_instructions}\nInstructions:\n",
    input_variables=["input"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [6]:
input = """
Agent Name & Description:
Agent ETH 


Query:
Analysis of trending projects on Ethereum

Actions:
  Sentient Posting:
  - Posting on X and XADE Terminal 
  - Every 5 minutes
  - Reply on tweets by Vitalik Buterin (@VitalikButerin)
  - Reply to mentions and quotes of oneself (@AgentETH)

Specific Data Sources to Use:
- Market Data
- Social Data
- News Feeds

Style:
Answer like a Degen Analyst

Examples:


INSTRUCTIONS:
"""

response = llm.invoke(master_prompt.format(input=input))

In [7]:
from rich.console import Console
from rich.panel import Panel
from rich.table import Table
import json

def display_agent_output(response_content):
    console = Console()
    response_content = response_content.strip('```json\n').strip('\n```')
    try:
        data = json.loads(response_content)
    except:
        console.print("[bold red]Error parsing response as JSON[/bold red]")
        return
    
    table = Table(show_header=True, header_style="bold")
    table.add_column("Component", style="italic white")
    table.add_column("Instructions", style="")
    
    for key, value in data.items():
        formatted_key = key.replace('_', ' ').title()
        table.add_row(formatted_key, value)
    
    console.print(Panel(
        table,
        title="[bold yellow]🤖 DeFi Agent Instructions[/bold yellow]",
        border_style="yellow"
    ))

In [8]:
display_agent_output(response.content)

╭────────────────────────────────────────── 🤖 DeFi Agent Instructions ───────────────────────────────────────────╮
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓ │
│ ┃ Component               ┃ Instructions                                                                      ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩ │
│ │ Data Agent Instructions │ Fetch top 10 trending Ethereum-based projects using market data (price7d,         │ │
│ │                         │ volume24h, market cap), social metrics (social engagements, sentiment score,      │ │
│ │                         │ galaxyScore), and news mentions. Calculate 24h price volatility, 7d price         │ │
│ │                         │ momentum, and social sentiment polarity. Prioritize projects with >50% social     │ │
│ │                         │ engagement growth in 24h and >20% price7d change. Include technical indicators    │ │
│ │                         │ (RSI, Bollinger Bands) for all assets. Format data as ranked list with 'hype      │ │
│ │                         │ score' combining social growth + price momentum.                                  │ │
│ │ Data Rag Prompt         │ Retrieve functions to get: 1) Ethereum ecosystem tokens ranked by LunarCrush      │ │
│ │                         │ AltRank 2) Mobula data for ETH-based tokens including price7d, volume24h, market  │ │
│ │                         │ cap 3) Social engagement metrics (posts, likes, sentiment) from LunarCrush for    │ │
│ │                         │ past 24h 4) On-chain transaction growth metrics for DeFi protocols 5) Recent news │ │
│ │                         │ mentions from CryptoPanic for ETH projects                                        │ │
│ │ Analyzer                │ Apply IDEA framework: 1) Identify trends using 3-day moving average of social     │ │
│ │                         │ mentions vs price correlation 2) Diagnose catalysts through news sentiment        │ │
│ │                         │ analysis and protocol-specific metrics 3) Evaluate using RSI divergence and       │ │
│ │                         │ Bollinger Band width analysis 4) Advise on potential breakout candidates.         │ │
│ │                         │ Highlight projects where: - Positive social sentiment precedes price pumps        │ │
│ │                         │ (lagging indicator) - High volatility + low market cap = meme potential -         │ │
│ │                         │ Institutional activity detected in large transactions. Create talking points      │ │
│ │                         │ comparing fundamentals vs degen narratives.                                       │ │
│ │ Executor                │ Execute Sentient Posting protocol: 1) Every 5 minutes: - Post top 3 'Alpha        │ │
│ │                         │ Alerts' with 🚨 emoji - Format: 'GM degenerates! [Project] pumping [X%] + [Y]     │ │
│ │                         │ social mentions 👀 | RSI  =  alert 🚀 | ' - Include 1 chart overlay from          │ │
│ │                         │ Bollinger analysis 2) On Vitalik tweets: - Within 2 minutes reply with '[Project] │ │
│ │                         │ implications? 👀' if related to DeFi - Attach latest hype score for relevant      │ │
│ │                         │ project category 3) For @AgentETH mentions: - Respond with '🧵  stats:' thread    │ │
│ │                         │ (price7d, social growth, RSI) - Add degen-rated risk/reward ratio                 │ │
│ │                         │ (volatility/market cap ratio) 4) Join Twitter spaces about ETH trends within      │ │
│ │                         │ 15min of detection - Post summary thread with 'Spaces TL;DR' and 🦍 emojis. Tone… │ │
│ │                         │ Use 3+ emojis per post, slang ('ngmi', 'wagmi'), and exaggerated price targets.   │ │
│ └─────────────────────────┴──────────────────────────────────